In [53]:
import requests
import json
import keys

#Add a file keys.py in the folder, and input your own API KEYS

class WiseClient:
    def __init__(self, access_token):
        self.access_token = access_token
        self.base_url = "https://api.sandbox.transferwise.tech/"
        self.headers = {
            'Authorization': f'Bearer {self.access_token}',
            'Content-Type': 'application/json'
        }
    def generate_uuid(self):
        url = "https://www.uuidgenerator.net/api/guid"

        payload={}
        headers = {}

        response = requests.request("GET", url, headers=headers, data=payload)

        return response.text

        
    def get_balance(self, profile_id):
        url = self.base_url + f'v1/borderless-accounts?profileId={profile_id}'
        response = requests.get(url, headers=self.headers)
        balance_data = response.json()
        return balance_data  # This should return your account balances in different currencies

    def create_quote(self, profile_id, source_currency, target_currency, amount):
        url = self.base_url + 'v1/quotes'
        quote_payload = {
            'profile': profile_id,
            'source': source_currency,
            'target': target_currency,
            'sourceAmount': amount,
            'rateType': 'FIXED',
            'type': 'BALANCE_CONVERSION'
        }
        response = requests.post(url, headers=self.headers, data=json.dumps(quote_payload))
        quote_data = response.json()
        return quote_data  # This should return the details of the quote including its id

    def check_id(self,account_id):
        url = self.base_url+ '/v1/accounts/' + account_id
        response = requests.get(url, headers=self.headers)
        return response.json()

    def initiate_transfer(self, target_account_id, quote_id, reference):
        url = self.base_url + 'v1/transfers'
        transfer_payload = {
            'targetAccount': target_account_id,
            'quote': quote_id,
            'customerTransactionId': self.generate_uuid(),
            'details': {
                'reference': reference,
                'transferPurpose': 'verification.transfers.purpose.pay.bills'
            }
        }
        response = requests.post(url, headers=self.headers, data=json.dumps(transfer_payload))
        transfer_data = response.json()
        return transfer_data  
    
    def make_payment(self, profile_id, transfer_id):
        url = self.base_url + f'v3/profiles/{profile_id}/transfers/{transfer_id}/payments'
        payment_payload = {
            'type': 'BALANCE',
        }
        response = requests.post(url, headers=self.headers, data=json.dumps(payment_payload))
        payment_data = response.json()
        return payment_data
    



In [28]:
wise_client = WiseClient(keys.API)

profile_id = keys.PROFILE_ID
balances = wise_client.get_balance(profile_id)

# Find USD and EUR balance ids
usd_balance_id = next((balance['id'] for balance in balances[0]['balances'] if balance['currency'] == 'USD'), None)
eur_balance_id = next((balance['id'] for balance in balances[0]['balances'] if balance['currency'] == 'EUR'), None)

print(usd_balance_id,eur_balance_id)

158818 158817


In [49]:
# https://github.com/transferwise/api-docs/blob/master/source/includes/_payouts.md

In [54]:
# Create a Wise client
wise_client = WiseClient(keys.API)

# Create a quote
profile_id = keys.PROFILE_ID # replace this with your profile id
quote_data = wise_client.create_quote(profile_id, 'USD', 'EUR', 100)

balances = wise_client.get_balance(profile_id)

# Find USD and EUR balance ids
usd_balance_id = next((balance['id'] for balance in balances[0]['balances'] if balance['currency'] == 'USD'), None)
eur_balance_id = next((balance['id'] for balance in balances[0]['balances'] if balance['currency'] == 'EUR'), None)

# Initiate a transfer
target_account_id = keys.RECIPIENT_ID
reference = 'reference'  # replace this with your reference text
transfer_data = wise_client.initiate_transfer(target_account_id, quote_data['id'], reference)

payment_data = wise_client.make_payment(profile_id, transfer_data['id'])

print(payment_data)

{'type': 'BALANCE', 'status': 'REJECTED', 'errorCode': 'balance.payment-option-unavailable', 'errorMessage': None}
